<a href="https://colab.research.google.com/github/AkankshaB123/leet-code/blob/main/Consecutive_streak_preferred_approach_wexample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3-ways to solve a consecutive streak question:


1.  reg. Window Function
* Lag() for prev_date. Remember this: grouping by user_id, and created group (for streak)
*   Refer to existing consecutive q's already solved on github for this (Using row number or sum() over(unbounded preceding and current row) to create streak groups)

2.   reg. Without Window Function


*  Not Exists logic + end_date>= start_date (to arrive at streak length)
*  Edge case missed: multiple streaks per user will generate cross-join; should use row_number to eliminate those.
**

# ***With Approach 1.***

In [3]:
#Example Data
import pandas as pd
import sqlite3

data = [
    ("A", "2024-01-01"),
    ("A", "2024-01-02"),
    ("A", "2024-01-03"),
    ("A", "2024-01-05"),
    ("B", "2024-02-10"),
    ("B", "2024-02-12"),
    ("B", "2024-02-13"),
]

df = pd.DataFrame(data, columns=["user_id", "activity_date"])
df["activity_date"] = pd.to_datetime(df["activity_date"])

In [4]:
#2.Load data into SQLite (in-memory)
conn = sqlite3.connect(":memory:")
df.to_sql("activity", conn, index=False, if_exists="replace")

7

In [17]:
sql1 = """
WITH lagged AS (
    SELECT
        user_id,
        activity_date,
        LAG(activity_date) OVER (
            PARTITION BY user_id
            ORDER BY activity_date
        ) AS prev_dt
    FROM activity
),

flags AS (
    SELECT
        user_id,
        activity_date,
        prev_dt,
        CASE
            WHEN prev_dt IS NULL THEN 1
            WHEN julianday(activity_date) - julianday(prev_dt) = 1 THEN 0
            ELSE 1
        END AS streak_break
    FROM lagged
)
    SELECT
        *,
        SUM(streak_break) OVER (
            PARTITION BY user_id
            ORDER BY activity_date
            ROWS UNBOUNDED PRECEDING
        ) AS streak_id
    FROM flags

"""
result1 = pd.read_sql(sql1, conn)
result1

,user_id,activity_date,prev_dt,streak_break,streak_id
0,A,2024-01-01 00:00:00,None,1,1
1,A,2024-01-02 00:00:00,2024-01-01 00:00:00,0,1
2,A,2024-01-03 00:00:00,2024-01-02 00:00:00,0,1
3,A,2024-01-05 00:00:00,2024-01-03 00:00:00,1,2
4,B,2024-02-10 00:00:00,None,1,1
5,B,2024-02-12 00:00:00,2024-02-10 00:00:00,1,2
6,B,2024-02-13 00:00:00,2024-02-12 00:00:00,0,2


# ***Note(for above case):***   

1.   streak_break = 1 means "start a new streak"
2.   streak_break = 0 means "continue the same streak"

In [18]:
sql = """
WITH lagged AS (
    SELECT
        user_id,
        activity_date,
        LAG(activity_date) OVER (
            PARTITION BY user_id
            ORDER BY activity_date
        ) AS prev_dt
    FROM activity
),

flags AS (
    SELECT
        user_id,
        activity_date,
        prev_dt,
        CASE
            WHEN prev_dt IS NULL THEN 1
            WHEN julianday(activity_date) - julianday(prev_dt) = 1 THEN 0
            ELSE 1
        END AS streak_break
    FROM lagged
),

streaks AS (
    SELECT
        user_id,
        activity_date,
        streak_break,
        SUM(streak_break) OVER (
            PARTITION BY user_id
            ORDER BY activity_date
            ROWS UNBOUNDED PRECEDING
        ) AS streak_id
    FROM flags
)

SELECT
    user_id,
    MIN(activity_date) AS streak_start,
    MAX(activity_date) AS streak_end,
    COUNT(*) AS streak_length
FROM streaks
GROUP BY user_id, streak_id
ORDER BY user_id, streak_start;
"""
result = pd.read_sql(sql, conn)
result


,user_id,streak_start,streak_end,streak_length
0,A,2024-01-01 00:00:00,2024-01-03 00:00:00,3
1,A,2024-01-05 00:00:00,2024-01-05 00:00:00,1
2,B,2024-02-10 00:00:00,2024-02-10 00:00:00,1
3,B,2024-02-12 00:00:00,2024-02-13 00:00:00,2
